In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import confusion_matrix

# I. SPLITTING DATA
We want to make sure to incorporate **SUBSAMPLING** to make help with the lopsided distribution

In [3]:
def subsample(x, y, keep=1, num:int = 20000 ):
    '''
    Subsample the dataset by keeping v number of rows for each value k on the y variables
    :param x: np array or  DF of the feature matrix
    :param y: np array or  DF of the output array 
    :param s: dict of (k,v) where k is the value of y and v is the number of rows to keep
    '''
    #First split the file
    x_train, x_test, y_train, y_test = train_test_split(x,y)
    print("Original :\nx_train:{}, x_test:{}, y_train:{}, y_test:{}".format(x_train.shape,x_test.shape,y_train.shape,y_test.shape))
    print("There are {} positive samples in the original train set!".format(sum(y_train)))
    print("There are {} positive samples in the original train set!".format(sum(y_test)))
    #Keep the positives
    y_keep = y_train[y_train == keep]
    x_keep = x_train[y_train == keep]    
    #print("Keep", y_keep.shape, x_keep.shape)
    #Subsample the negatives
    y_sample = y_train[y_train != keep].sample(num)
    x_sample = x_train.loc[y_sample.index]
    #Reappend the files
    y_train = pd.concat((y_sample, y_keep), axis=0)
    x_train = pd.concat((x_sample, x_keep), axis=0)
    #Shuffle the train sets together
    y_train = y_train.sample(frac=1)
    x_train = x_train.loc[y_train.index]
    print("Subsample :\nx_train:{}, y_train:{}, x_test:{}, y_test:{}".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))
    return x_train, x_test, y_train, y_test

# II.RANDOM FOREST 

In [4]:
from sklearn.ensemble import RandomForestClassifier as RF

## DAILY DATA

In [5]:
d = pd.read_pickle("social_data_slide_scaled_d.pkl")
print("Daily data",d.shape)

Daily data (80940, 1066)


In [6]:
social_feat_d = list(pd.read_pickle("social_features_d.pkl"))
x_train, x_test, y_train, y_test = subsample(d[social_feat_d], d['pumped_yn'], num=2000)

Original :
x_train:(60705, 1056), x_test:(20235, 1056), y_train:(60705,), y_test:(20235,)
There are 334 positive samples in the original train set!
There are 92 positive samples in the original train set!
Subsample :
x_train:(2334, 1056), y_train:(2334,), x_test:(20235, 1056), y_test:(20235,)


In [7]:
x_train.head()

,code_repo_stars_d0,code_repo_stars_d1,code_repo_stars_d2,code_repo_stars_d3,code_repo_stars_d4,code_repo_stars_d5,code_repo_stars_d6,code_repo_stars_d7,code_repo_stars_d8,code_repo_stars_d9,code_repo_stars_d10,code_repo_stars_d11,code_repo_stars_d12,code_repo_stars_d13,code_repo_stars_d14,code_repo_stars_d15,code_repo_stars_d16,code_repo_stars_d17,code_repo_stars_d18,code_repo_stars_d19,code_repo_stars_d20,code_repo_stars_d21,code_repo_stars_d22,code_repo_stars_d23,code_repo_stars_d24,code_repo_stars_d25,code_repo_stars_d26,code_repo_stars_d27,code_repo_stars_d28,code_repo_stars_d29,code_repo_stars_d30,code_repo_stars_d31,code_repo_forks_d0,code_repo_forks_d1,code_repo_forks_d2,code_repo_forks_d3,code_repo_forks_d4,code_repo_forks_d5,code_repo_forks_d6,code_repo_forks_d7,code_repo_forks_d8,code_repo_forks_d9,code_repo_forks_d10,code_repo_forks_d11,code_repo_forks_d12,code_repo_forks_d13,code_repo_forks_d14,code_repo_forks_d15,code_repo_forks_d16,code_repo_forks_d17,code_repo_forks_d18,code_repo_forks_d19,code_repo_forks_d20,code_repo_forks_d21,code_repo_forks_d22,code_repo_forks_d23,code_repo_forks_d24,code_repo_forks_d25,code_repo_forks_d26,code_repo_forks_d27,code_repo_forks_d28,code_repo_forks_d29,code_repo_forks_d30,code_repo_forks_d31,code_repo_subscribers_d0,code_repo_subscribers_d1,code_repo_subscribers_d2,code_repo_subscribers_d3,code_repo_subscribers_d4,code_repo_subscribers_d5,code_repo_subscribers_d6,code_repo_subscribers_d7,code_repo_subscribers_d8,code_repo_subscribers_d9,code_repo_subscribers_d10,code_repo_subscribers_d11,code_repo_subscribers_d12,code_repo_subscribers_d13,code_repo_subscribers_d14,code_repo_subscribers_d15,code_repo_subscribers_d16,code_repo_subscribers_d17,code_repo_subscribers_d18,code_repo_subscribers_d19,code_repo_subscribers_d20,code_repo_subscribers_d21,code_repo_subscribers_d22,code_repo_subscribers_d23,code_repo_subscribers_d24,code_repo_subscribers_d25,code_repo_subscribers_d26,code_repo_subscribers_d27,code_repo_subscribers_d28,code_repo_subscribers_d29,code_repo_subscribers_d30,code_repo_subscribers_d31,code_repo_contributors_d0,code_repo_contributors_d1,code_repo_contributors_d2,code_repo_contributors_d3,code_repo_contributors_d4,code_repo_contributors_d5,code_repo_contributors_d6,code_repo_contributors_d7,code_repo_contributors_d8,code_repo_contributors_d9,code_repo_contributors_d10,code_repo_contributors_d11,code_repo_contributors_d12,code_repo_contributors_d13,code_repo_contributors_d14,code_repo_contributors_d15,code_repo_contributors_d16,code_repo_contributors_d17,code_repo_contributors_d18,code_repo_contributors_d19,code_repo_contributors_d20,code_repo_contributors_d21,code_repo_contributors_d22,code_repo_contributors_d23,code_repo_contributors_d24,code_repo_contributors_d25,code_repo_contributors_d26,code_repo_contributors_d27,code_repo_contributors_d28,code_repo_contributors_d29,code_repo_contributors_d30,code_repo_contributors_d31,code_repo_open_pull_issues_d0,code_repo_open_pull_issues_d1,code_repo_open_pull_issues_d2,code_repo_open_pull_issues_d3,code_repo_open_pull_issues_d4,code_repo_open_pull_issues_d5,code_repo_open_pull_issues_d6,code_repo_open_pull_issues_d7,code_repo_open_pull_issues_d8,code_repo_open_pull_issues_d9,code_repo_open_pull_issues_d10,code_repo_open_pull_issues_d11,code_repo_open_pull_issues_d12,code_repo_open_pull_issues_d13,code_repo_open_pull_issues_d14,code_repo_open_pull_issues_d15,code_repo_open_pull_issues_d16,code_repo_open_pull_issues_d17,code_repo_open_pull_issues_d18,code_repo_open_pull_issues_d19,code_repo_open_pull_issues_d20,code_repo_open_pull_issues_d21,code_repo_open_pull_issues_d22,code_repo_open_pull_issues_d23,code_repo_open_pull_issues_d24,code_repo_open_pull_issues_d25,code_repo_open_pull_issues_d26,code_repo_open_pull_issues_d27,code_repo_open_pull_issues_d28,code_repo_open_pull_issues_d29,code_repo_open_pull_issues_d30,code_repo_open_pull_issues_d31,code_repo_closed_pull_issues_d0,code_repo_closed_pull_issues_d1,code_repo_closed_

In [80]:
rf = RF()
params_rf = {'n_estimators':[50,100,200,500]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}

In [81]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv= 5, scoring="roc_auc")
CV_rfc.fit(x_train, y_train)

KeyboardInterrupt: 

In [73]:
CV_rfc.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [64]:
rf_best = RF(criterion = 'entropy', max_depth = 50, max_features='sqrt', n_estimators=200)
rf_best.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=50, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

....Predict on __training__ set..

In [65]:
confusion_matrix(y_train,rf_best.predict(x_train)).ravel()

array([4999,    1,   38,  284])

...Predict on __testing__ ...

In [66]:
tn, fp, fn, tp = confusion_matrix(y_test,rf_best.predict(x_test)).ravel()

In [67]:
print(tn,fp,fn,tp)

20041 90 99 5


In [260]:
del tn, fp, fn, tp

## HOURLY MODEL

In [55]:
h = pd.read_pickle("social_data_slide_scaled_h.pkl") 
social_feat_h = list(pd.read_pickle("social_features_h.pkl"))
print( "Hourly data",h.shape)

Hourly data (80940, 1627)


In [56]:
rf = RF()
x_train, x_test, y_train, y_test = subsample(h[social_feat_h], h['pumped_yn'], num=2000)

Original :
x_train:(60705, 1617), x_test:(20235, 1617), y_train:(60705,), y_test:(20235,)
There are 323 positive samples in the original train set!
There are 103 positive samples in the original train set!
Subsample :
x_train:(2323, 1617), y_train:(2323,), x_test:(20235, 1617), y_test:(20235,)


In [57]:
rf = RF()
params_rf = {'n_estimators':[50,100,200]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}

In [58]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv= 5, scoring="roc_auc")
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [48]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 100}

In [49]:
rf_best = RF(criterion = 'entropy', max_depth = 10, max_features='sqrt', n_estimators=100)
rf_best.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

...Predict on training set...

In [50]:
confusion_matrix(y_train,rf_best.predict(x_train)).ravel()

array([5000,    0,  300,   26])

...Predict on testing set...

In [52]:
tn, fp, fn, tp = confusion_matrix( y_test,rf_best.predict(x_test)).ravel()

In [53]:
print(tn,fp,fn,tp)

20135 0 100 0


In [54]:
del tn, fp, fn, tp